In [1]:
%cd yolov8_DeepSORT

[WinError 2] The system cannot find the file specified: 'yolov8_DeepSORT'
E:\Juypter\Computer-Vision-Projects\Child and Adult detection


# Libraries

In [2]:
import torch as th
from ultralytics import YOLO
import time
import cv2
import numpy as np
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

# DeepSORT

In [3]:
# Initialize DeepSort tracker with custom parameters
deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(
    model_path=deep_sort_weights, 
    max_dist=0.2, 
    min_confidence=0.3, 
    max_iou_distance=0.7, 
    max_age=70, 
    n_init=3, 
    nn_budget=100
)

# Video input and output details

In [4]:
# Define the video path
video_path = 'Training Videos/downloaded_video_8.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'output_no_skip_7_fixed.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


# Load YOLO model

In [5]:
model = YOLO('best.pt')

# Generating result

In [6]:
# Initialize variables
unique_track_ids = set()
confirmed_tracks = set()  # To track only confirmed person count
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()

# Main loop without skipping frames
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Video finished or error in reading frame.")
        break

    # Prepare the frame for detection
    og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = og_frame.copy()

    # Perform detection with YOLO model, limiting to class indices 0 (Adult) and 1 (Child)
    results = model(frame, conf=0.5, classes=[0, 1])

    # Ensure there are detection results before accessing them
    if results and results[0].boxes:
        for result in results:
            boxes = result.boxes
            cls = boxes.cls.tolist()
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh.cpu().numpy()

        # Update tracker with detections
        tracks = tracker.update(bboxes_xywh, conf, og_frame)

        # Filter valid tracks and draw bounding boxes
        for track in tracker.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue

            track_id = track.track_id
            unique_track_ids.add(track_id)

            # Get bounding box coordinates
            x1, y1, x2, y2 = track.to_tlbr()
            w = x2 - x1
            h = y2 - y1

            # Assign class label based on class index (0 for 'Adult', 1 for 'Child')
            if len(pred_cls) > 0:
                # Ensure the current track's index does not go out of bounds
                cls_index = min(track_id, len(pred_cls) - 1)
                if pred_cls[cls_index] == 1:
                    class_label = 'Child'
                    color = (255, 0, 0)  # Blue for Child
                elif pred_cls[cls_index] == 0:
                    class_label = 'Adult'
                    color = (0, 0, 255)  # Red for Adult
                else:
                    class_label = "Class Unknown"
                    color = (0, 255, 0)  # Green for unknown class
            else:
                class_label = "Class Unknown"
                color = (0, 255, 0)

            confirmed_tracks.add(track_id)

            # Draw bounding box and ID on the frame
            cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)
            cv2.putText(og_frame, f"{class_label}-{track_id}", (int(x1) + 10, int(y1) - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

    # Update FPS calculation
    current_time = time.perf_counter()
    elapsed = current_time - start_time
    counter += 1
    if elapsed > 1:
        fps = counter / elapsed
        counter = 0
        start_time = current_time

    # Display confirmed person count and FPS on the frame
    person_count = len(confirmed_tracks)
    cv2.putText(og_frame, f"People Count: {person_count}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(og_frame, f"FPS: {int(fps)}", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Write the processed frame to the output video
    out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

# Release resources properly
cap.release()
out.release()



0: 160x256 (no detections), 30.1ms
Speed: 1.0ms preprocess, 30.1ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.0ms
Speed: 1.0ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 21.3ms
Speed: 1.0ms preprocess, 21.3ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.7ms
Speed: 1.0ms preprocess, 20.7ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 18.0ms
Speed: 2.0ms preprocess, 18.0ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 22.0ms
Speed: 1.0ms preprocess, 22.0ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.3ms
Speed: 1.0ms preprocess, 20.3ms i


0: 160x256 1 Adult, 1 Child, 19.0ms
Speed: 1.4ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 19.3ms
Speed: 1.3ms preprocess, 19.3ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 19.3ms
Speed: 1.0ms preprocess, 19.3ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.5ms
Speed: 1.0ms preprocess, 18.5ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 19.0ms
Speed: 1.1ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 19.0ms
Speed: 1.7ms preprocess, 19.0ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.0ms
Speed: 1.1ms preprocess, 18.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.0ms
Speed: 1.0ms preprocess, 

Speed: 3.7ms preprocess, 19.7ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 16.8ms
Speed: 0.0ms preprocess, 16.8ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.5ms
Speed: 0.0ms preprocess, 20.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.2ms
Speed: 0.0ms preprocess, 20.2ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 11.9ms
Speed: 5.0ms preprocess, 11.9ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 17.6ms
Speed: 1.0ms preprocess, 17.6ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 5.8ms
Speed: 3.0ms preprocess, 5.8ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 15.3ms
Speed: 0.0ms preprocess, 15.3ms inference, 0.0ms postprocess per image 

Speed: 1.1ms preprocess, 20.2ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 88.4ms
Speed: 0.0ms preprocess, 88.4ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.3ms
Speed: 1.0ms preprocess, 18.3ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.8ms
Speed: 1.0ms preprocess, 18.8ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 16.5ms
Speed: 1.0ms preprocess, 16.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 19.4ms
Speed: 1.0ms preprocess, 19.4ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 16.0ms
Speed: 0.0ms preprocess, 16.0ms inference, 1.0ms postprocess p


0: 160x256 2 Adults, 1 Child, 18.0ms
Speed: 1.0ms preprocess, 18.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 2 Adults, 1 Child, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 2 Adults, 1 Child, 18.7ms
Speed: 0.0ms preprocess, 18.7ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.0ms
Speed: 1.4ms preprocess, 18.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 19.8ms
Speed: 1.0ms preprocess, 19.8ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 18.5ms
Speed: 0.0ms preprocess, 18.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 17.5ms
Speed: 1.0ms preprocess, 17.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.1ms
Speed: 1.0ms preprocess, 18.1ms inferenc

...


0: 160x256 (no detections), 19.3ms
Speed: 1.5ms preprocess, 19.3ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.1ms
Speed: 0.0ms preprocess, 20.1ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 19.5ms
Speed: 1.1ms preprocess, 19.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 20.2ms
Speed: 1.0ms preprocess, 20.2ms inference, 0.6ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 21.1ms
Speed: 0.0ms preprocess, 21.1ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 19.4ms
Speed: 1.0ms preprocess, 19.4ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 19.9ms
Speed: 1.1ms preprocess, 19.9ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 21.5ms
Speed: 1.0ms preprocess, 21.5ms i


0: 160x256 1 Adult, 1 Child, 20.5ms
Speed: 1.0ms preprocess, 20.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 21.6ms
Speed: 1.5ms preprocess, 21.6ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 21.4ms
Speed: 1.0ms preprocess, 21.4ms inference, 0.3ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 20.0ms
Speed: 1.0ms preprocess, 20.0ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 19.7ms
Speed: 0.0ms preprocess, 19.7ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.5ms
Speed: 1.2ms preprocess, 18.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.5ms
Speed: 0.0ms preprocess, 18.5ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 18.6ms
Speed: 1.0ms preprocess, 


0: 160x256 1 Adult, 20.3ms
Speed: 0.0ms preprocess, 20.3ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 21.2ms
Speed: 1.0ms preprocess, 21.2ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 21.4ms
Speed: 1.0ms preprocess, 21.4ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 19.2ms
Speed: 1.0ms preprocess, 19.2ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 18.9ms
Speed: 0.8ms preprocess, 18.9ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 21.5ms
Speed: 0.0ms preprocess, 21.5ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 (no detections), 21.5ms
Speed: 1.0ms preprocess, 21.5ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 21.3ms
Speed: 0.3ms preprocess, 21.3ms inference, 1.0ms postprocess per image at shape (


0: 160x256 1 Child, 18.6ms
Speed: 1.2ms preprocess, 18.6ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 18.1ms
Speed: 1.5ms preprocess, 18.1ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 20.1ms
Speed: 1.2ms preprocess, 20.1ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 20.0ms
Speed: 1.1ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 19.6ms
Speed: 1.3ms preprocess, 19.6ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 19.6ms
Speed: 0.0ms preprocess, 19.6ms inference, 0.6ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 20.1ms
Speed: 0.0ms preprocess, 20.1ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Child, 19.6ms
Speed: 1.0ms preprocess, 19.6ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)


Speed: 1.3ms preprocess, 20.5ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 21.6ms
Speed: 1.0ms preprocess, 21.6ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 20.6ms
Speed: 1.1ms preprocess, 20.6ms inference, 2.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 20.1ms
Speed: 1.0ms preprocess, 20.1ms inference, 1.5ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 21.6ms
Speed: 1.0ms preprocess, 21.6ms inference, 0.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 20.6ms
Speed: 0.0ms preprocess, 20.6ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 21.5ms
Speed: 0.5ms preprocess, 21.5ms inference, 0.6ms postprocess per image at shape (1, 3, 160, 256)

0: 160x256 1 Adult, 1 Child, 21.3ms
Speed: 0.0ms preprocess, 21.3ms inference, 1.0ms postprocess p